In [369]:
import numpy as np
import pandas as pd
import gzip
import json

from pprint import pprint

In [370]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Osama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [371]:
train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [372]:
train_classification_df.groupby("category").count()

,user_id
category,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [373]:
username2_category["kod8net"]

'tech'

In [374]:
'''train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]




# Load the additional CSV file
additional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"
additional_data_df = pd.read_csv(additional_data_path)

# Extract and rename the relevant columns
additional_data_df = additional_data_df[['Unnamed: 0', 'influencerCategory']].rename(columns={
    'Unnamed: 0': 'user_id', 
    'influencerCategory': 'category'
})

additional_data_df = additional_data_df.dropna()



# Convert the 'category' column to string type and apply .lower(), handling any NaN or unexpected values
additional_data_df['category'] = additional_data_df['category'].astype(str).fillna('').apply(str.lower)

# Append the new data to the original train_classification_df
train_classification_df = pd.concat([train_classification_df, additional_data_df], ignore_index=True)

# Update the username2_category dictionary with the new data
username2_category.update(additional_data_df.set_index("user_id").to_dict()["category"])

# Check the updated data by viewing the first few rows
print(train_classification_df.head())

# Re-check the category distribution
train_classification_df.groupby("category").count()'''


'train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)\ntrain_classification_df = train_classification_df.rename(columns={\'Unnamed: 0\': \'user_id\', \'label\': \'category\'})\n\n# Unifying labels\ntrain_classification_df["category"] = train_classification_df["category"].apply(str.lower)\nusername2_category = train_classification_df.set_index("user_id").to_dict()["category"]\n\n\n\n\n# Load the additional CSV file\nadditional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"\nadditional_data_df = pd.read_csv(additional_data_path)\n\n# Extract and rename the relevant columns\nadditional_data_df = additional_data_df[[\'Unnamed: 0\', \'influencerCategory\']].rename(columns={\n    \'Unnamed: 0\': \'user_id\', \n    \'influencerCategory\': \'category\'\n})\n\nadditional_data_df = additional_data_df.dropna()\n\n\n\n# Convert the \'category\' column to string type and apply .lower(), handling any NaN or unexpec

In [375]:
train_data_path = "/Users/Osama/Downloads/CS412PROJ/training-dataset.jsonl.gz"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with gzip.open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]
    if username in username2_category:
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile

In [376]:
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

train_profile_df.shape

(2741, 44)

In [377]:
test_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,None,1265,665,True,False,...,None,None,PERSONAL_BLOG,False,False,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,None,28025,4,True,False,...,None,None,ENERGY_COMPANY,False,False,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [378]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import emoji


def preprocess_text(text: str):
    

    # lower casing Turkish Text, Don't use str.lower :)
    text = text.casefold()

    #text = emoji.demojize(text, delimiters=(" ", " "))

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters and punctuation
    # HERE THE EMOJIS stuff are being removed, you may want to keep them :D
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


corpus = []

# to keep the label order
train_usernames = []

for username, posts in username2posts_train.items():
  train_usernames.append(username)

  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)


  # joining the posts of each user with a \n
  user_post_captions = "\n".join(cleaned_captions)
  corpus.append(user_post_captions)



#custom_stopwords = list(set(turkish_stopwords).union({
 #   'the', 'and', 'with', 'for', 'you', 'to', 'of', 'in', 'our', 'your', 'is', 'are','bir'
#}))
vectorizer = TfidfVectorizer(stop_words=turkish_stopwords, max_features=10000,min_df=10,ngram_range=(1, 3))

# fit the vectorizer
vectorizer.fit(corpus)

# transform the data into vectors
x_post_train = vectorizer.transform(corpus)
y_train = [username2_category.get(uname, "NA") for uname in train_usernames]
feature_names = vectorizer.get_feature_names_out()

# Inspect the frequency of each word
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)

# Show the most frequent words (words in many posts)
print(df_tfidf.sum().sort_values(ascending=False).head(30))




test_usernames = []
test_corpus = []
for username, posts in username2posts_test.items():
  test_usernames.append(username)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

  user_post_captions = "\n".join(cleaned_captions)
  test_corpus.append(user_post_captions)


# Just transforming! No Fitting!!!!!
x_post_test = vectorizer.transform(test_corpus)

bir            192.474290
the            106.220712
and             65.960927
to              59.345598
of              59.225437
olsun           56.723444
yeni            46.531255
istanbul        46.083638
in              44.908546
olarak          43.520030
olan            41.210234
with            39.918309
kutlu           39.892111
kutlu olsun     39.315072
iyi             37.609096
güzel           37.207437
kadar           36.265992
you             35.986349
for             34.862311
özel            32.566009
bilgi           32.489645
devam           32.122644
var             31.813758
our             31.544601
kasım           31.361084
günü            31.188695
birlikte        30.770312
büyük           30.195825
ilk             29.995598
teşekkür        29.513940
dtype: float64


In [379]:
# Making sure everything is fine
assert y_train.count("NA") == 0


In [380]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['ab', 'abant', 'abd', ..., 'şıklık', 'şıklığı', 'şımartın'],
      dtype=object)

In [381]:
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)
df_tfidf.head(2)

,ab,abant,abd,abdulkadir,abdullah,abi,abiye,abone,about,about the,...,şöyle,şükran,şükranla,şükranlarımızı,şükür,şık,şık bir,şıklık,şıklığı,şımartın
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.042156,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [382]:
df_tfidf.shape

(2741, 10000)

In [383]:
print("Shape of df_tfidf:", df_tfidf.shape)
print("Length of y_train:", len(y_train))

Shape of df_tfidf: (2741, 10000)
Length of y_train: 2741


In [418]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime
def extract_features(posts):
    features = []
    for post in posts:
        caption = post.get("caption", "")
        comments_count = post.get("comments_count", 0)
        like_count = post.get("like_count", 0)
        media_type = post.get("media_type", "UNKNOWN")
        timestamp = post.get("timestamp", "")
        
        # Process timestamp
        if timestamp:
            dt = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
            hour = dt.hour
            day = dt.day
            month = dt.month
        else:
            hour = day = month = 0
        
        # Feature list
        features.append({
            "caption": caption,
            "comments_count": comments_count,
            "media_type": media_type,
            "hour": hour,
            "day": day,
            "month": month,
            "like_count": like_count  # Target variable
        })
    return features


train_features = []
train_targets = []

for username, posts in username2posts_train.items():
    user_features = extract_features(posts)
    for feature in user_features:
        train_features.append(feature)
        train_targets.append(feature["like_count"])

# Create a DataFrame for easy manipulation
train_df = pd.DataFrame(train_features)

# Step 3: Feature Transformation
# One-hot encode `media_type`
vectorizer = TfidfVectorizer(max_features=10000, stop_words=turkish_stopwords,min_df=10,ngram_range=(1, 3))

onehot_encoder = OneHotEncoder()
media_type_encoded = onehot_encoder.fit_transform(train_df[["media_type"]])

# Apply TF-IDF vectorization to captions
train_df["caption"] = train_df["caption"].fillna("")

# Apply TF-IDF vectorization to captions
caption_tfidf = vectorizer.fit_transform(train_df["caption"])

# Combine features into a single matrix
from scipy.sparse import hstack
final_features = hstack([
    caption_tfidf, 
    media_type_encoded, 
    train_df[["comments_count", "hour", "day", "month"]].values
])


In [421]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(final_features, y_train, test_size=0.2, stratify=y_train,random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [94824, 2192]

In [385]:
x_train.shape

(2192, 10000)

In [386]:
x_val.shape

(549, 10000)

In [387]:
#@title Test Data
test_data_path = "/Users/Osama/Downloads/CS412PROJ/test-classification-round1.dat"

with open(test_data_path, "rt") as fh:
    for i, line in enumerate(fh):
        print(line.strip())
        if i >= 4:  # Stop after 5 lines
            break

print("*****")

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

print(test_unames[:5])

ozhotelstr
elleturkiye
sozerinsaatorhangazi
sanliurfapiazzaavym
rusanozden
*****
['ozhotelstr', 'elleturkiye', 'sozerinsaatorhangazi', 'sanliurfapiazzaavym', 'rusanozden']


In [388]:
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = train_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)


test_unames.remove("screenname")

screenname


In [389]:
df_test = pd.DataFrame(np.array(x_test), columns=feature_names)
df_test.head(2)

,ab,abant,abd,abdulkadir,abdullah,abi,abiye,abone,about,about the,...,şöyle,şükran,şükranla,şükranlarımızı,şükür,şık,şık bir,şıklık,şıklığı,şımartın
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.007865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.024666,0.0,0.0,0.0,0.0,0.010521,0.0,0.0,0.0,0.0


In [413]:
from sklearn.preprocessing import OneHotEncoder
from datetime import datetime

def extract_features(posts):
    features = []
    for post in posts:
        caption = post.get("caption", "")
        comments_count = post.get("comments_count", 0)
        like_count = post.get("like_count", 0)
        media_type = post.get("media_type", "UNKNOWN")
        timestamp = post.get("timestamp", "")
        
        # Process timestamp
        if timestamp:
            dt = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
            hour = dt.hour
            day = dt.day
            month = dt.month
        else:
            hour = day = month = 0
        
        # Feature list
        features.append({
            "caption": caption,
            "comments_count": comments_count,
            "media_type": media_type,
            "hour": hour,
            "day": day,
            "month": month,
            "like_count": like_count  # Target variable
        })
    return feature

train_features = []
train_targets = []
for username, posts in username2posts_train.items():
    user_features = extract_features(posts)
    for feature in user_features:
        train_features.append(feature)
        train_targets.append(feature["like_count"])

# Create DataFrame
train_df = pd.DataFrame(train_features)

# Encode `media_type` as one-hot
onehot_encoder = OneHotEncoder(sparse=False)
media_type_encoded = onehot_encoder.fit_transform(train_df[["media_type"]])

# TF-IDF on captions
vectorizer = TfidfVectorizer(max_features=5000, stop_words=turkish_stopwords)
caption_tfidf = vectorizer.fit_transform(train_df["caption"])

# Combine features
from scipy.sparse import hstack
final_features = hstack([
    caption_tfidf, 
    media_type_encoded, 
    train_df[["comments_count", "hour", "day", "month"]].values
])

In [414]:
def predict_like_count(username, current_post=None):
  def get_avg_like_count(posts:list):
    total = 0.
    for post in posts:
      if current_post is not None and post["id"] == current_post["id"]:
        continue

      like_count = post.get("like_count", 0)
      if like_count is None:
        like_count = 0
      total += like_count

    if len(posts) == 0:
      return 0.

    return total / len(posts)

  if username in username2posts_train:
    return get_avg_like_count(username2posts_train[username])
  elif username in username2posts_test:
    return get_avg_like_count(username2posts_test[username])
  else:
    print(f"No data available for {username}")
    return -1

In [391]:
def log_mse_like_counts(y_true, y_pred):
  """
  Calculate the Log Mean Squared Error (Log MSE) for like counts (log(like_count + 1)).

  Parameters:
  - y_true: array-like, actual like counts
  - y_pred: array-like, predicted like counts

  Returns:
  - log_mse: float, Log Mean Squared Error
  """
  # Ensure inputs are numpy arrays
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)

  # Log transformation: log(like_count + 1)
  log_y_true = np.log1p(y_true)
  log_y_pred = np.log1p(y_pred)

  # Compute squared errors
  squared_errors = (log_y_true - log_y_pred) ** 2

  # Return the mean of squared errors
  return np.mean(squared_errors)

In [392]:
#@title Train Dataset evaluation

y_like_count_train_true = []
y_like_count_train_pred = []
for uname, posts in username2posts_train.items():
  for post in posts:
    pred_val = predict_like_count(uname, post)
    true_val = post.get("like_count", 0)
    if true_val is None:
      true_val = 0

    y_like_count_train_true.append(true_val)
    y_like_count_train_pred.append(pred_val)

print(f"Log MSE Train= {log_mse_like_counts(y_like_count_train_true, y_like_count_train_pred)}")

Log MSE Train= 1.2271047744059362


In [393]:
#@title Test Dataset

path = "/Users/Osama/Downloads/CS412PROJ/test-regression-round1.jsonl"
output_path = "/Users/Osama/Downloads/CS412PROJ/testy-regression-round1.jsonl"

to_predict_like_counts_usernames = []
output_list = []
with open(path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    # let's predict
    pred_val = predict_like_count(sample["username"])
    sample["like_count"] = int(pred_val)
    output_list.append(sample)

with open(output_path, "wt") as of:
  json.dump(output_list, of)




In [394]:
pprint(output_list[:3])

[{'caption': 'KOZA 2023 2.si Damla’nın koleksiyonu, Latincede ‘Memento Mori’ '
             'olarak bilinen ‘ölümlü olduğunu hatırla’ anlamındaki ifadeden '
             'esinleniyor. Koleksiyon, hayatın ve ölümün, para, işçi, kral ve '
             'kraliçe kavramları üzerinden yaratıcı görünümlerle bir araya '
             'getirilmesini amaçlıyor. Ölüm sembollerinden esinlenen desenler '
             'kullanan Damla, “kağıt parçasından ibaret olmak” kavramını '
             'vurguluyor. Koleksiyon, yaşamın ve ölümün aynı anda ifade '
             'edilmesini hedefliyor; kırmızı ve mavi ışıklarla veya '
             'gözlüklerle görülen hologram efekti kullanılarak bu konsept '
             'sahneye taşınıyor. Kırmızı renk ölümü, mavi ise yaşamı '
             'simgeliyor. Koleksiyon, ofis giyimlerinden esinlenerek '
             'kravatlar, gömlekler ve evrak çantaları içeriyor. Klasik sivri '
             'burun çizmelerin üzerine spor ayakkabıların üst yüzeyi '
             'yerle

In [395]:
'''from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor


# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize the regressor
rf = RandomForestRegressor(random_state=42)

# Perform grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid_search.fit(x_train, y_train)

# Best parameters and best model
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_'''


'from sklearn.model_selection import GridSearchCV\nfrom sklearn.ensemble import RandomForestRegressor\n\n\n# Define the parameter grid\nparam_grid = {\n    \'n_estimators\': [50, 100, 200],\n    \'max_depth\': [None, 10, 20, 30],\n    \'min_samples_split\': [2, 5, 10],\n    \'min_samples_leaf\': [1, 2, 4],\n    \'max_features\': [\'auto\', \'sqrt\', \'log2\']\n}\n\n# Initialize the regressor\nrf = RandomForestRegressor(random_state=42)\n\n# Perform grid search\ngrid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring=\'neg_mean_squared_error\', verbose=2, n_jobs=-1)\ngrid_search.fit(x_train, y_train)\n\n# Best parameters and best model\nprint("Best Parameters:", grid_search.best_params_)\nbest_model = grid_search.best_estimator_'

In [396]:
'''from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Prepare the target variable: Average like count per user
user_avg_likes = {}
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    user_avg_likes[username] = avg_likes

# Create the target array
y_like_counts = [user_avg_likes[uname] for uname in train_usernames]
y_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]

# Train-test split
x_train, x_val, y_train, y_val = train_test_split(
    x_post_train, y_like_counts, test_size=0.2, random_state=42
)

# Train an MLP Regressor
xgb_regressor = XGBRegressor(
    n_estimators=100,   # Number of trees
    learning_rate=0.1,  # Step size for updating weights
    max_depth=6,        # Depth of each tree
    random_state=42
)

# Train the model
xgb_regressor.fit(x_train, y_train)

y_val = np.maximum(y_val, 0)



# Predict on validation set
y_pred_val = xgb_regressor.predict(x_val)
y_pred_val = np.maximum(y_pred_val, 0)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
print(f"MSE on validation set: {mse}")

# Predict on test set
y_test_pred = xgb_regressor.predict(x_post_test)


# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred
})
print(test_predictions.head())'''


'from xgboost import XGBRegressor\nfrom sklearn.neural_network import MLPRegressor\nfrom sklearn.metrics import mean_squared_error\nfrom sklearn.model_selection import train_test_split\nimport numpy as np\nimport pandas as pd\n\n# Prepare the target variable: Average like count per user\nuser_avg_likes = {}\nfor username, posts in username2posts_train.items():\n    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])\n    user_avg_likes[username] = avg_likes\n\n# Create the target array\ny_like_counts = [user_avg_likes[uname] for uname in train_usernames]\ny_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]\n\n# Train-test split\nx_train, x_val, y_train, y_val = train_test_split(\n    x_post_train, y_like_counts, test_size=0.2, random_state=42\n)\n\n# Train an MLP Regressor\nxgb_regressor = XGBRegressor(\n    n_estimators=100,   # Number of trees\n    learning_rate=0.1,  # Step size for updating weights\n    max_depth=6,        # Depth of each tree

In [410]:
import statsmodels.api as sm
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Prepare the target variable: Average like count per user
user_avg_likes = {}
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    user_avg_likes[username] = avg_likes

# Create the target array
y_like_counts = [user_avg_likes[uname] for uname in train_usernames]
y_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]

# Train-test split
x_train, x_val, y_train, y_val = train_test_split(
    x_post_train, y_like_counts, test_size=0.2, random_state=42
)

lgbm_regressor = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.2,
    max_depth=30,
    min_data_in_leaf=30,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    objective="poisson",  # Poisson objective for count data
    random_state=42
)

# Train the model
lgbm_regressor.fit(x_train, y_train)

# Predict on validation set
y_pred_val = lgbm_regressor.predict(x_val)

# Ensure non-negative predictions
y_pred_val = np.maximum(y_pred_val, 0)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
print(f"MSE on validation set: {mse}")

# Predict on test set
y_test_pred = lgbm_regressor.predict(x_post_test)

# Ensure non-negative predictions
y_test_pred = np.maximum(y_test_pred, 0)

# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred
})

print(test_predictions.head())

C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.109845 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 210149
[LightGBM] [

In [398]:
print(test_predictions.head(50))

                      username  predicted_like_count
0               beyazyakaliyiz           9599.277991
1   totalenergies_istasyonlari             61.663299
2                  konforyatak             24.042859
3                     ht_kulup            169.114373
4                    ajansspor            344.821231
5         yusufelibelediyesi08             73.046840
6                     4bros.tr            104.646842
7              groundy.kadikoy            455.485480
8                drtubagunebak            308.385433
9                   nihatcan11             62.915241
10               mustafaaakcay            266.759324
11                  mtmofamily           9031.409849
12                 bonitoperde            133.375867
13     balkanlardangelenlezzet             15.089956
14      pedagoganne__gulozturk            516.564634
15                      iktm34            292.218342
16                    yudumyag            246.160252
17      bulentozdemir.edebiyat             53.

In [399]:
if np.isnan(y_val).any() or np.isnan(y_pred_val).any():
    print("NaN values found in y_val or y_pred_val!")


In [412]:
log_errors = np.log1p(y_val) - np.log1p(y_pred_val)  # Use np.log1p to handle zero values gracefully

# Compute the absolute log errors
abs_log_errors = np.abs(log_errors)

# Sum of log errors
sum_log_errors = np.sum(abs_log_errors)

print(f"Sum of Log Errors: {sum_log_errors}")

Sum of Log Errors: 728.0061700155054


In [401]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.15],
    'max_depth': [5, 10, -1],
    'min_data_in_leaf': [10, 20, 30],
    'feature_fraction': [0.8, 0.9],
    'bagging_fraction': [0.8, 0.9]
}

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=lgbm_regressor, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

print("Best parameters found: ", grid_search.best_params_)


KeyboardInterrupt: 

In [78]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Prepare the target variable: Average like count per user
user_avg_likes = {}
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    user_avg_likes[username] = avg_likes

# Create the target array
y_like_counts = [user_avg_likes[uname] for uname in train_usernames]
y_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]
# Train-test split
x_train, x_val, y_train, y_val = train_test_split(
    x_post_train, y_like_counts, test_size=0.2, random_state=42
)

xgb_regressor = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    objective='reg:squarederror',  # Default; can change to 'reg:poisson' for count data
    random_state=42
)

# Train the model
xgb_regressor.fit(x_train, y_train)

# Predict on validation set
y_pred_val = xgb_regressor.predict(x_val)

# Post-process: Clamp negative predictions to 0
y_pred_val = np.maximum(y_pred_val, 0)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
log_errors = np.log1p(y_val) - np.log1p(y_pred_val)  # Use np.log1p to handle zero values
abs_log_errors = np.abs(log_errors)
sum_log_errors = np.sum(abs_log_errors)

print(f"MSE on validation set: {mse}")
print(f"Sum of Log Errors: {sum_log_errors}")

# Predict on test set
y_test_pred = xgb_regressor.predict(x_post_test)

# Post-process: Clamp negative predictions to 0
y_test_pred = np.maximum(y_test_pred, 0)

# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred
})

print(test_predictions.head())

# Save test predictions to a CSV file
#test_predictions.to_csv("test_predictions.csv", index=False)


C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


MSE on validation set: 2819676604.812782
Sum of Log Errors: 1278.0898952402472
                     username  predicted_like_count
0              beyazyakaliyiz          10770.258789
1  totalenergies_istasyonlari            453.646881
2                 konforyatak            370.162567
3                    ht_kulup            484.007294
4                   ajansspor           2237.495117


In [79]:
log_errors = np.log1p(y_val) - np.log1p(y_pred_val)  # Use np.log1p to handle zero values gracefully

# Compute the absolute log errors
abs_log_errors = np.abs(log_errors)

# Sum of log errors
sum_log_errors = np.sum(abs_log_errors)

print(f"Sum of Log Errors: {sum_log_errors}")

Sum of Log Errors: 1278.0898952402472


In [36]:
# Check for NaN in user_avg_likes
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    if np.isnan(avg_likes):
        print(f"NaN detected for user: {username}, posts: {posts}")
    user_avg_likes[username] = avg_likes


NameError: name 'user_avg_likes' is not defined

In [28]:
'''# Calculate log errors
log_errors = np.abs(y_val_reg - y_val_pred_reg)

# Sum of log errors
sum_log_errors = np.sum(log_errors)

print("Sum of Log Errors:", sum_log_errors)
'''

NameError: name 'y_val_pred_reg' is not defined